# **PnL Data Pipeline**

*By Simran Bains*

---

### Table of Contents
1. [Input Data 🗒](#input-data-)
2. [Parsing, Cleaning & Transformations 🧽](#parsing-cleaning--transformations-)
3. [Value at Risk (VaR) Calculation 🔢](#value-at-risk-var-calculation-)
4. [Output Results and Data Storage 📈](#output-results-and-data-storage-)


In [26]:
#Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

#Additional
from google.colab import files
from datetime import datetime
import os

##1. Input Data 🗒

In [2]:
uploaded = files.upload()

Saving data_engineer_sample_data.csv to data_engineer_sample_data (2).csv


In [3]:
df = pd.read_csv("data_engineer_sample_data.csv", header=None)
df.head()

,0,1,2,3
0,TradeID,PortfolioId,businessDate,PnL
1,T123,101,10/1/2024 14:30,1500.5
2,T123,102,10/1/2024,2000.75
3,T123,101,10/2/2024 9:15,-500.25
4,T124,102,10/1/2024,300


In [4]:
#Column identification
df.columns = ["TradeID", "PortfolioId", "businessDate", "PnL"]
print("Original Columns:", df.columns)

Original Columns: Index(['TradeID', 'PortfolioId', 'businessDate', 'PnL'], dtype='object')


In [5]:
#Renaming columns to meet specification requirements
df.rename(columns={"PortfolioId": "BookId", "businessDate": "PnLDate"}, inplace=True)

In [6]:
#Converting PnLDate to string format
df["PnLDate"] = pd.to_datetime(df["PnLDate"], errors="coerce").dt.strftime("%Y-%m-%d")

#Ensuring numeric variable for BookID
df["BookId"] = pd.to_numeric(df["BookId"], errors="coerce")

#Ensuring float variable for PnL
df["PnL"] = pd.to_numeric(df["PnL"], errors="coerce")

print(df)

     TradeID  BookId     PnLDate       PnL
0    TradeID     NaN         NaN       NaN
1       T123   101.0  2024-10-01   1500.50
2       T123   102.0  2024-10-01   2000.75
3       T123   101.0  2024-10-02   -500.25
4       T124   102.0  2024-10-01    300.00
..       ...     ...         ...       ...
175     T207   106.0  2024-12-29  13150.75
176     T208   107.0  2024-12-30  13250.50
177     T208   108.0  2024-12-30 -13350.25
178     T209   109.0  2024-12-31  13450.00
179     T209   110.0  2024-12-31 -13550.75

[180 rows x 4 columns]


<ipython-input-6-f65e8e67292e>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["PnLDate"] = pd.to_datetime(df["PnLDate"], errors="coerce").dt.strftime("%Y-%m-%d")


## 2. Parsing, Cleaning & Transformations 🧽

In [7]:
#Removing duplicated header
df = df.iloc[1:].reset_index(drop=True)

In [8]:
df.head()

,TradeID,BookId,PnLDate,PnL
0,T123,101.0,2024-10-01,1500.50
1,T123,102.0,2024-10-01,2000.75
2,T123,101.0,2024-10-02,-500.25
3,T124,102.0,2024-10-01,300.00
4,T124,103.0,2024-01-10,0.00


In [9]:
# Add a column with the processing date
df["Processed"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [10]:
df.head()

,TradeID,BookId,PnLDate,PnL,Processed
0,T123,101.0,2024-10-01,1500.50,2025-02-02 11:36:25
1,T123,102.0,2024-10-01,2000.75,2025-02-02 11:36:25
2,T123,101.0,2024-10-02,-500.25,2025-02-02 11:36:25
3,T124,102.0,2024-10-01,300.00,2025-02-02 11:36:25
4,T124,103.0,2024-01-10,0.00,2025-02-02 11:36:25


## 3. Value at Risk (VaR) Calculation 🔢

i. Aggregate the PnL values by PnL date

In [11]:
df_agg = df.groupby("PnLDate")["PnL"].sum().reset_index()

ii. Sort the aggregated values in ascending order

In [12]:
df_agg = df_agg.sort_values(by="PnL")

iii. Find the position in the aggregated data associated with the chosen confidence level

In [16]:
def position_var(data, confidence_level):
    pos_var = int(np.ceil((1 - confidence_level) * len(data))) - 1
    return data.iloc[pos_var]

iv. VaR is the value associated with this position

In [22]:
# a. Calculate VaR for every trade at the 95% confidence level
var_95_conf = df.groupby("TradeID")["PnL"].apply(lambda x: position_var(x.sort_values(), 0.95))
print(var_95_conf)

TradeID
T123     -500.25
T124        0.00
T125     -250.50
T126     -100.00
T127     -600.50
          ...   
T205   -12750.75
T206    12850.50
T207   -13050.00
T208   -13350.25
T209   -13550.75
Name: PnL, Length: 87, dtype: float64


The VaR at 95% confidence level per trade shows the maximum expected loss for each trade under normal market conditions. Some trades (e.g., T123) have negative VaR values, indicating expected potential losses. A few trades have a VaR of 0.00, suggesting that those trades did not experience significant adverse price movements.

In [23]:
# b. Calculate VaR for every trade at the 97.5% confidence level
var_975_conf = df.groupby("TradeID")["PnL"].apply(lambda x: position_var(x.sort_values(), 0.975))
print(var_975_conf)

TradeID
T123     -500.25
T124        0.00
T125     -250.50
T126     -100.00
T127     -600.50
          ...   
T205   -12750.75
T206    12850.50
T207   -13050.00
T208   -13350.25
T209   -13550.75
Name: PnL, Length: 87, dtype: float64


The VaR at 97.5% confidence level provides a stricter threshold than the 95% level. Generally, the values are lower (more negative) than the 95% level, reflecting higher potential losses when accounting for a more extreme market scenario.

In [24]:
# c. Calculate VaR at the 95% confidence level across all trades
var_95_all_trade = position_var(df["PnL"].sort_values(), 0.95)
print(var_95_all_trade)

-11450.0


The VaR at 99% per book aggregates risk at a higher level, showing the worst potential losses for each book of trades. This helps determine which books are at the highest risk and may require further hedging or diversification. Larger absolute VaR values indicate books that are more volatile or hold riskier positions.

In [25]:
# d. VaR at the 99% confidence level per book
var_99_book = df.groupby("BookId")["PnL"].apply(lambda x: position_var(x.sort_values(), 0.99))
print(var_99_book)

BookId
101.0   -10650.00
102.0   -12750.75
103.0    -5850.00
104.0   -11950.75
105.0   -13050.00
106.0   -11150.75
107.0   -12250.00
108.0   -13350.25
109.0   -11450.00
110.0   -13550.75
Name: PnL, dtype: float64


The overall 95% VaR across all trades represents the portfolio-wide risk exposure. A significantly large negative value suggests a considerable risk across all trades. If this number is relatively small, it implies diversification effects where losses from some trades are offset by gains in others.

## 4. Output Results and Data Storage 📈

The output and data storage section follows a structured Medallion Architecture approach to store and organise data at different processing stages. This ensures data integrity, accessibility and efficient risk reporting.

The bronze layer stores the cleaned raw data after initial parsing and transformations. This ensures the cleaned data is available for further transformations and risk calculations.

In [36]:
#Medallion Pattern: Bronze Results
bronze_results = "/content/bronze/cleaned_data.csv"
os.makedirs(os.path.dirname(bronze_results), exist_ok=True)
df.to_csv(bronze_results, index=False)

The silver layer stores intermediate analytical results i.e., the computed Value at Risk (VaR) metrics. This allows easier access to processed and structured risk analytics for further analysis.

In [37]:
#Medallion Pattern: Silver Results
silver_results = "/content/silver/"
os.makedirs(silver_results, exist_ok=True)
var_95_conf.to_csv(os.path.join(silver_results, "var_95_trade.csv"), header=True)
var_975_conf.to_csv(os.path.join(silver_results, "var_975_trade.csv"), header=True)
pd.DataFrame({"var_95_all_trade": [var_95_all_trade]}).to_csv(os.path.join(silver_results, "var_95_all_trade.csv"), index=False)
var_99_book.to_csv(os.path.join(silver_results, "var_99_book.csv"), header=True)

The gold layer stores a high level summary report linking key risk metrics to their storage locations. This simplifies reporting and traceability and ensures for analysis in respective file locations.

In [42]:
#Medallion Pattern: Gold Results
gold_results = "/content/gold/summary_report.csv"
os.makedirs(os.path.dirname(gold_results), exist_ok=True)
mp_results = {
    "Metric": ["VaR_95_Per_Trade", "var_975_trade", "var_95_all_trade", "var_99_book"],
    "Location": [
        os.path.join(silver_results, "var_95_trade.csv"),
        os.path.join(silver_results, "var_975_trade.csv"),
        os.path.join(silver_results, "var_95_all_trade.csv"),
        os.path.join(silver_results, "var_99_book.csv")
    ]
}
pd.DataFrame(mp_results).to_csv(gold_results, index=False)

In [41]:
print("Medallion Pattern Results:")
print(pd.DataFrame(mp_results))

Medallion Pattern Results:
             Metric                              Location
0  VaR_95_Per_Trade      /content/silver/var_95_trade.csv
1     var_975_trade     /content/silver/var_975_trade.csv
2  var_95_all_trade  /content/silver/var_95_all_trade.csv
3       var_99_book       /content/silver/var_99_book.csv


In [39]:
!ls /content/
!ls /content/bronze/
!ls /content/silver/
!ls /content/gold/

 bronze				     'data_engineer_sample_data (2).csv'   Documents   sample_data
'data_engineer_sample_data (1).csv'   data_engineer_sample_data.csv	   gold        silver
cleaned_data.csv
var_95_all_trade.csv  var_95_trade.csv	var_975_trade.csv  var_99_book.csv
summary_report.csv


In [40]:
from google.colab import files
files.download('/content/gold/summary_report.csv')
!zip -r pnl_task_results.zip /content/
files.download('pnl_task_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.01.30/ (stored 0%)
  adding: content/.config/logs/2025.01.30/14.18.57.126926.log (deflated 57%)
  adding: content/.config/logs/2025.01.30/14.18.44.811352.log (deflated 87%)
  adding: content/.config/logs/2025.01.30/14.18.57.806410.log (deflated 56%)
  adding: content/.config/logs/2025.01.30/14.18.46.676701.log (deflated 58%)
  adding: content/.config/logs/2025.01.30/14.18.35.872284.log (deflated 58%)
  adding: content/.config/logs/2025.01.30/14.18.13.5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>